# Introduction: Book Recommendation System

In this notebook, we will build a book recommendation system based on all the books articles on Wikipedia and neural network embeddings of books and links. This is a great look at the potential for neural networks to create meaningful embeddings of high dimensional data and one practical application of deep learning.

## Read in Data

The data is stored as json with line for every book. This data contains every single book article on Wikipedia which was parsed in the Downloading and Parsing Wikipedia Data Notebook.

In [5]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [1]:
import json

books = []

with open('found_books_filtered.ndjson', 'r') as fin:
    # Append each line to the books
    books = [json.loads(l) for l in fin]


print(f'Found {len(books)} books.')

Found 37040 books.


Each book contains the title, the information from the `Infobox book` template, the internal wikipedia links, the external links, the date of last edit, and the number of characters in the article (a rough estimate of the length of the article).

In [2]:
books[15]

['Derech Mitzvosecha',
 {'name': 'Derech Mitzvosecha (Sefer Hamitzvos)',
  'image': 'Derech Mitzvosecha 2014-05-08 01-34.jpg',
  'image_size': '200px',
  'caption': 'Derech Mitzvosecha, 1912 edition',
  'author': 'Rabbi Menachem Mendel Schneersohn ( " Tzemach Tzedek " ), the third Rebbe of Chabad',
  'genre': 'non-fiction',
  'isbn': '0826655904',
  'published': '7x10 Hardcover, 570pp, (Kehot Publication Society, Brooklyn New York)',
  'subject': 'Jewish mysticism, Chabad philosophy',
  'language': 'Hebrew'},
 ['Mitzvos',
  'Menachem Mendel Schneersohn',
  'Rebbe',
  'Chabad',
  'Hasidic',
  'Chabad philosophy',
  'Tzitzit',
  'Tefillin',
  'Hillel the Elder',
  'Poltava',
  'Kehot Publication Society',
  'Eliyahu Touger',
  'Sichos in English',
  'Tanya',
  'Shneur Zalman of Liadi',
  'Category:Chabad-Lubavitch (Hasidic dynasty)',
  'Category:Books about Judaism',
  'Category:Chabad-Lubavitch texts'],
 ['http://www.books.google.com/books?id=HYLi3l9ylWwC',
  'http://www.store.kehotonli

# Map Books to Integers

First we want to create a mapping of books to integers. When we feed books into the neural network, we will have to represent them as numbers, so this mapping will let us keep track of the books.

In [7]:
book_index = {book[0]: idx for idx, book in enumerate(books)}
index_book = {idx: book for book, idx in book_index.items()}

book_index['War and Peace']
index_book[22475]

22475

'War and Peace'

# Number of Unique Wikilinks

First we can do a little data exploration. Let's find the number of unique Wikilinks (links to other Wikipedia articles) and the most common ones. To create a single list from a list of lists, we can use the `itertools` chain method.

In [35]:
from itertools import chain

wikilinks = list(chain(*[book[2] for book in books]))
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 315703 unique wikilinks.


How many of these are links to other books? 

In [36]:
wikilinks_other_books = [link for link in wikilinks if link in book_index.keys()]
print(f"There are {len(set(wikilinks_other_books))} unique links to other books.")

There are 17036 unique links to other books.


Let's take a look at the counts of each wikilink. We'll make a utility function that takes in a list and returns a sorted ordered dictionary of the counts of the items in the list.

In [37]:
from collections import Counter, OrderedDict

def count_items(l):
    """Return ordered dictionary of counts of objects in `l`"""
    # Create a counter object
    counts = Counter(l)
    
    # Sort by highest count first
    counts = sorted(counts.items(), key = lambda x: x[1], reverse = True)
    counts = OrderedDict(counts)
    
    return counts

In [38]:
wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

[('Hardcover', 7643),
 ('Paperback', 7484),
 ('Wikipedia:WikiProject Books', 6116),
 ('Wikipedia:WikiProject Novels', 6087),
 ('English language', 4308),
 ('The New York Times', 3960),
 ('United States', 3394),
 ('Science fiction', 3121),
 ('science fiction', 2661),
 ('Publishers Weekly', 2415)]

The most linked to pages are in fact not that surprising! One thing we should notice is that there are discrepancies in capitalization. We want to normalize across capitalization, so we'll lowercase all of the links and redo the counts.

In [39]:
wikilinks = [link.lower() for link in wikilinks]
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 301955 unique wikilinks.


In [40]:
wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

[('paperback', 8979),
 ('hardcover', 8856),
 ('wikipedia:wikiproject books', 6116),
 ('wikipedia:wikiproject novels', 6088),
 ('science fiction', 5917),
 ('english language', 4375),
 ('the new york times', 3976),
 ('united states', 3397),
 ('novel', 3045),
 ('publishers weekly', 2415)]

That actually changes the rankings! This illustrates an important point: make sure to take a look at your data before modeling! 

In [43]:
wikilinks_other_books = [link.lower() for link in wikilinks_other_books]
wikilink_books_counts = count_items(wikilinks_other_books)
list(wikilink_books_counts.items())[:10]

[('the discontinuity guide', 148),
 ('the encyclopedia of science fiction', 145),
 ('dracula', 72),
 ('the dresden files', 70),
 ('the encyclopedia of fantasy', 66),
 ('encyclopædia britannica', 61),
 ('the wonderful wizard of oz', 61),
 ('nineteen eighty-four', 59),
 ("alice's adventures in wonderland", 54),
 ('don quixote', 52)]

Since there are so many unique wikilinks, I'm going to limit the list to wikilinks mentioned 5 or more times. Hopefully this reduces the noise that might come from wikilinks that only appear a few times.

In [47]:
links = [t[0] for t in wikilink_counts.items() if t[1] >= 5]
print(len(links))
links[-10:]

38935


['catherine ryan hyde',
 'life with billy (book)',
 'shan states',
 'burmese chronicles#ramanya',
 'northern ndebele people',
 'international journal of hindu studies',
 'charles henderson (character)',
 'anarchy: a journal of desire armed',
 'american museum of fly fishing',
 'key of perihelion']

The pound sign in the link name means that the link directs to a specific section on the Wikipedia page. Although we could normalize these to the title of the wikipedia article, we'll leave the pound sign in for now.

## Wikilinks to Index

As with the books, we need to map the wikilinks to integers. We'll also create the reverse mapping.

In [50]:
link_index = {link: idx for idx, link in enumerate(links)}
index_link = {idx: link for link, idx in link_index.items()}

link_index['the new york times']
index_link[6]

6

'the new york times'

# Build a Training Set

In order for any machine learning model to learn, it needs a training set. We are going to treat this as a supervised learning problem: given an article, can the neural network determine whether or not a wikilink was present on that page? To do this, we need to create a dictionary of books and the associate links on their page. We already have this in the list of books, but now we need to change our data structure.

In [52]:
# book_link_mapping = {book_index[book[0]]: [link_index[link.lower()] for link in book[2] if link.lower() in links] for book in books}

# book_link_mapping = {}

# for i, book in enumerate(books):
#     book_link_mapping[book_index[book[0]]] = []
#     for link in book[2]:
#         if link.lower() in links:
#             book_link_mapping[book_index[book[0]]].append(link_index[link.lower()])
    
#     if i % 100 == 0:
#         print(i, end = '\r')

For each book and each wikilink in the books articles, we'll add it to a list of pairs with the form (book_index, link_index). We also filter out the links that did not have at least 5 occurrences.

In [53]:
pairs = []

for book in books:
    pairs.extend((book_index[book[0]], link_index[link.lower()]) for link in book[2] if link.lower() in links)
    
len(pairs), len(links), len(books)

(805304, 38935, 37040)

We now have 805304 positive examples on which to train! Each pair represents one internal wikipedia link for one book.

In [54]:
index_book[pairs[5000][0]], index_link[pairs[5000][1]]

("Doraemon: Nobita's Space Heroes", 'south korea')

In [55]:
index_book[pairs[10000][0]], index_link[pairs[10000][1]]

('KP: The Autobiography', 'hardcover')

The more common wikilinks might be less useful. In a way, this could be similar to problem that creates a need for TF-IDF: the more common a word, the less representative it is of the sentence.

## Generator For Training Samples

We want to generate positive samples and negative samples. The positive samples are simple: pick one random pair from the `pairs` and assign it a 1. The negative samples are also fairly easy: pick one random link and one random book, make sure they are not in the `pairs`, and assign them a -1.

In [65]:
import numpy as np
import random
random.seed(100)

def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0):
    """Generate batches of samples for training"""
    batch_size = n_positive * (1 + negative_ratio)
    
    batch = np.zeros((batch_size, 3))
    
    while True:
        # randomly choose positive examples
        for idx, (book_id, link_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (book_id, link_id, 1)
            
        idx = n_positive
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_book = random.randrange(len(books))
            random_link = random.randrange(len(links))
            
            # Check to make sure this is not a positive example
            if (random_book, random_link) not in pairs:
                batch[idx, :] = (random_book, random_link, -1)
                idx += 1
                
        np.random.shuffle(batch)
        yield {'book': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]

In [67]:
next(generate_batch(pairs, n_positive = 2, negative_ratio = 2))

({'book': array([21977., 29880., 20277.,  5792.,  3138.,  3812.]),
  'link': array([15053., 17263., 13360., 12904., 13401., 28269.])},
 array([-1., -1., -1.,  1., -1.,  1.]))

## Book Embedding Model

In [68]:
from keras.layers import Input, Embedding, Dot, Reshape
from keras.models import Model

In [69]:
def book_embedding_model(embedding_size = 50):
    book = Input(name = 'book', shape = [1])
    link = Input(name = 'link', shape = [1])
    
    book_embedding = Embedding(name = 'book_embedding',
                               input_dim = len(book_index),
                               output_dim = embedding_size)(book)
    
    link_embedding = Embedding(name = 'link_embedding',
                               input_dim = len(link_index),
                               output_dim = embedding_size)(link)
    
    book_embedding = Reshape(target_shape = [embedding_size])(book_embedding)
    link_embedding = Reshape(target_shape = [embedding_size])(link_embedding)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 1)([book_embedding, link_embedding])
    model = Model(inputs = [book, link], outputs = merged)
    model.compile(optimizer = 'Adam', loss = 'mse')
    return model

model = book_embedding_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
book (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
book_embedding (Embedding)      (None, 1, 50)        1852000     book[0][0]                       
__________________________________________________________________________________________________
link_embedding (Embedding)      (None, 1, 50)        1946750     link[0][0]                       
__________________________________________________________________________________________________
reshape_3 

# Train Model

The next step is just to train the model. We can fit the model on the generator and also make a generator for validation. 

In [ ]:
train_gen = generate_batch(pairs, n_positive=512, negative_ratio=5)
valid_gen = generate_batch(pairs, n_positive=512, negative_ratio=5)

model.fit_generator(train_gen, epochs = 20, 
                    steps_per_epoch = len(pairs) // 512,
                    validation_data = valid_gen, 
                    validation_steps = len(pairs) // 4,
                    verbose = 2)

Epoch 1/20


## Make Predictions

Of course the fun part is making predictions with the model. We can feed it any book and get the closest and furthest away books in the embedding space. 